## Gaussian Elimiation 

#### Importing Libraries

In [1]:
import numpy as np
import sympy 

#### Funtion Swap Rows


In [2]:
def swap_rows(M, row1, row2):
    M = M.copy() #copy matrix M so the changes not effect the originial M
    M[[row1,row2]] = M[[row2,row1]] #Swapping the indexes
    return M

let's practice with some examples. consider the following matrix:

In [3]:
M = np.array([
    [1,3,6],
    [0,-5,2],
    [-4,5,8]
    ])
print(M)

[[ 1  3  6]
 [ 0 -5  2]
 [-4  5  8]]


Swaping row 0 with row 2:

In [4]:
M_swapped = swap_rows(M,0,2)
print(M_swapped)

[[-4  5  8]
 [ 0 -5  2]
 [ 1  3  6]]


Finding the first non-zero value in a column starting from a specific value

In [5]:
def get_index_first_non_zero_value_from_column(M, column, starting_row): #Retrieve the index of the first non-zero value in a specified column of the given matrix.
    #Get the column array starting from the specified row
    column_array = M[starting_row:, column]
    for i, val in enumerate(column_array):
        # Iterate over every value in the column array. 
        # To check for non-zero values, you must always use np.isclose instead of doing "val == 0".
        if not np.isclose(val, 0, atol = 1e-5):
            # If one non zero value is found, then adjust the index to match the correct index in the matrix and return it.
            index = i +starting_row
            return index
        # If no non-zero value is found below it, return -1.
    return -1        

Let's practice with this function. Consider the following matrix.

In [6]:
N = np.array([
[0, 5, -3 ,6 ,8],
[0, 6, 3, 8, 1],
[0, 0, 0, 0, 0],
[0, 0, 0 ,0 ,7],
[0, 2, 1, 0, 4]
])
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


If you search for a value below the first column starting at the first row, the function should return -1:

In [7]:
print(get_index_first_non_zero_value_from_column(N,column = 0,starting_row = 0))

-1


In [8]:
print(get_index_first_non_zero_value_from_column(N,column = 1,starting_row = 1))

1


In [9]:
print(get_index_first_non_zero_value_from_column(N,column = -1,starting_row = 2))

3


Find the first non zero element for any row

In [10]:
def get_index_first_none_zero(M, row, augmented = False):
    # Create a copy to avoid modifying the original matrix
    M=M.copy()
     # If it is an augmented matrix, then ignore the constant values
    if augmented == True:
        # Isolating the coefficient matrix (removing the constant terms)
        M =M[:,:-1]
    # Get the desired row# Get the desired row    
    row_array = M[row]
    for i, val in enumerate(row_array):
        # If finds a non zero value, returns the index. Otherwise returns -1.
        if not np.isclose(val, 0, atol = 1e-5):
            return i

    return -1        

Let's practice with the same matrix as before:

In [11]:
print(N)

[[ 0  5 -3  6  8]
 [ 0  6  3  8  1]
 [ 0  0  0  0  0]
 [ 0  0  0  0  7]
 [ 0  2  1  0  4]]


In [12]:
print(f'Output for row 2: {get_index_first_none_zero(N, 2)}')
print(f'Output for row 3: {get_index_first_none_zero(N, 3)}')

Output for row 2: -1
Output for row 3: 4


<p>Now, let's pass the argument augmented = True. This will make the algorithm consider  𝑁
  an augmented matrix, therefore the last column will be removed from consideration. Now, the output for row 3 (starting from 0) should be different, excluding the last column, the output should be -1 as well, since in the coefficient matrix (the matrix without the last column) there is no non-zero element:</p>

In [13]:
print(f'Output for row 3: {get_index_first_none_zero(N, 3, augmented = True)}')

Output for row 3: -1


#### Constructing the Augmented Matrix

In [14]:
def augmented_matrix(A,B):
    augmented_M = np.hstack((A,B))
    return augmented_M

In [15]:
A = np.array([[1,2,3],[3,4,5],[4,5,6]])
B = np .array([[1],[5],[7]])

print(augmented_matrix(A,B)) 

[[1 2 3 1]
 [3 4 5 5]
 [4 5 6 7]]


In [16]:
def row_echelon_form(A, B):
    det_A = np.linalg.det(A)
    if np.isclose(det_A, 0) == True:
        return 'Singular System'
    
    A = A.copy()
    B = B.copy()

    A = A.astype('float64')
    B = B.astype('float64')

    num_rows = len(A)

    M = augmented_matrix(A,B)
    for row in range(num_rows):
        pivot_candidate = M[row,row]
        if np.isclose(pivot_candidate,0) == True:
            first_non_zero_value_below_pivot_candidate = get_index_first_non_zero_value_from_column(M,row,row)
            M = swap_rows(M, row, first_non_zero_value_below_pivot_candidate)
            pivot =M[row,row]
        else:
            pivot = pivot_candidate
        M[row] = (1/pivot) * M[row]

        for j in range(row + 1, num_rows):

            value_below_pivot = M[j,row]
            M[j] = M[j] - value_below_pivot*M[row]
            
    return M                

In [17]:
A = np.array([[1,2,3],[0,1,0], [0,0,5]])
B = np.array([[1], [2], [4]])
row_echelon_form(A,B)

array([[1. , 2. , 3. , 1. ],
       [0. , 1. , 0. , 2. ],
       [0. , 0. , 1. , 0.8]])

In [18]:
def back_substitution(M):
    M = M.copy()
    num_rows = M.shape[0]
    for row in reversed(range(num_rows)):
        substitution_row = M[row]
        index = get_index_first_none_zero(M,row,augmented=True)
        for j in range(row):
            row_to_reduce = M[j]
            value = row_to_reduce[index]
            row_to_reduce = row_to_reduce - (value * substitution_row)
            M[j,:] = row_to_reduce
        solution =M[:,-1]
        return solution    

In [19]:
def gaussian_elimination(A,B):
    row_echelon_M = row_echelon_form(A,B)
    if isinstance(row_echelon_M, str) and row_echelon_M == 'Singular System':
        return 'Singular System'
    solution = back_substitution(row_echelon_M)
    return solution
    

In [20]:
from sympy import symbols, Eq, Matrix, solve, sympify
def string_to_augmented_matrix(equations):
    # Split the input string into individual equations
    equation_list = equations.split('\n')
    equation_list = [x for x in equation_list if x != '']
    # Create a list to store the coefficients and constants
    coefficients = []
    
    ss = ''
    for c in equations:
        if c in 'abcdefghijklmnopqrstuvwxyz':
            if c not in ss:
                ss += c + ' '
    ss = ss[:-1]
    
    # Create symbols for variables x, y, z, etc.
    variables = symbols(ss)
    # Parse each equation and extract coefficients and constants
    for equation in equation_list:
        # Remove spaces and split into left and right sides
        sides = equation.replace(' ', '').split('=')
        
        # Parse the left side using SymPy's parser
        left_side = sympify(sides[0])
        
        # Extract coefficients for variables
        coefficients.append([left_side.coeff(variable) for variable in variables])
        
        # Append the constant term
        coefficients[-1].append(int(sides[1]))

    # Create a matrix from the coefficients
    augmented_matrix = Matrix(coefficients)
    augmented_matrix = np.array(augmented_matrix).astype("float64")

    A, B = augmented_matrix[:,:-1], augmented_matrix[:,-1].reshape(-1,1)
    
    return ss, A, B


In [21]:
equations = """
2*x + 6*y + 6*w + 8*z = 1
5*x + 3*y + 6*w = -10
4*y - 5*w + 8*z = 8
4*w + 8*z = 9
"""

variables, A, B = string_to_augmented_matrix(equations)

sols = gaussian_elimination(A, B)

if not isinstance(sols, str):
    for variable, solution in zip(variables.split(' '),sols):
        print(f"{variable} = {solution:.4f}")
else:
    print(sols)

x = -4.5385
y = -1.0577
w = -0.2692
z = 1.2596
